# [SQL Interview Questions on Data Lemur - Hard](https://datalemur.com/questions?difficulty=Hard&category=SQL)

##### Solved by: Dorothy Kunth

### 1. [Active User Retention - Facebook](https://datalemur.com/questions/user-retention)

Assume you're given a table containing information on Facebook user actions. Write a query to obtain number of monthly active users (MAUs) in July 2022, including the month in numerical format "1, 2, 3".

Hint:

An active user is defined as a user who has performed actions such as 'sign-in', 'like', or 'comment' in both the current month and the previous month.

``user_actions`` table

| user_id | event_id | event_type | event_date          |
|---------|----------|------------|---------------------|
| 445     | 7765     | sign-in    | 05/31/2022 12:00:00 |
| 445     | 3634     | like       | 06/05/2022 12:00:00 |
| 648     | 3124     | like       | 06/18/2022 12:00:00 |
| 648     | 2725     | sign-in    | 06/22/2022 12:00:00 |
| 648     | 8568     | comment    | 07/03/2022 12:00:00 |
| 445     | 4363     | sign-in    | 07/05/2022 12:00:00 |
| 445     | 2425     | like       | 07/06/2022 12:00:00 |
| 445     | 2484     | like       | 07/22/2022 12:00:00 |
| 648     | 1423     | sign-in    | 07/26/2022 12:00:00 |
| 445     | 5235     | comment    | 07/29/2022 12:00:00 |
| 742     | 6458     | sign-in    | 07/03/2022 12:00:00 |
| 742     | 1374     | comment    | 07/19/2022 12:00:00 |


### Solution
1. Use window function LAG() partition by user_id and order by event_date then extract the month

```sql
SELECT
    user_id,
    EXTRACT(MONTH FROM event_date) AS event_month,
    EXTRACT(MONTH FROM LAG(event_date) OVER (PARTITION BY user_id ORDER BY event_date)) AS previous_month,
    EXTRACT(MONTH FROM event_date) - 
        EXTRACT(MONTH FROM LAG(event_date) OVER (PARTITION BY user_id ORDER BY event_date)) AS diff
FROM user_actions
```
| user_id | event_month | previous_month | diff |
|---------|-------------|----------------|------|
| 445     | 5           | NULL           | NULL |
| 445     | 6           | 5              | 1    |
| 445     | 7           | 6              | 1    |
| 445     | 7           | 7              | 0    |
| 445     | 7           | 7              | 0    |
| 445     | 7           | 7              | 0    |
| 648     | 6           | NULL           | NULL |
| 648     | 6           | 6              | 0    |
| 648     | 7           | 6              | 1    |
| 648     | 7           | 7              | 0    |
| 742     | 7           | NULL           | NULL |
| 742     | 7           | 7              | 0    |


<br><br>
2. Use the above query as an inline query (FROM clause) and where event_month is 7 and diff is 1
```sql
SELECT
  event_month AS month,
  COUNT(DISTINCT user_id) AS monthly_active_users
FROM(
  SELECT
    user_id,
    EXTRACT(MONTH FROM event_date) AS event_month,
    EXTRACT(MONTH FROM LAG(event_date) OVER (PARTITION BY user_id ORDER BY event_date)) AS previous_month,
    EXTRACT(MONTH FROM event_date) - 
        EXTRACT(MONTH FROM LAG(event_date) OVER (PARTITION BY user_id ORDER BY event_date)) AS diff
  FROM user_actions) user_activity
WHERE event_month = 7 
  AND diff = 1
GROUP BY 1
```
| month | monthly_active_users |
|-------|----------------------|
| 7     | 2                    |


##### in progress